In [13]:
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Dense , Conv2D, UpSampling2D , BatchNormalization, Reshape , Flatten ,Dropout
from tensorflow.keras.layers import Activation, LeakyReLU , Input
from tensorflow.keras.callbacks import ModelCheckpoint ,  EarlyStopping
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# chap19 GAN -> 생성자와 판별자로 모델을 나눔
generator = Sequential()
generator.add(Dense(128*7*7, input_dim = 100,activation = LeakyReLU(0.2))) #  노이즈 이미지 생성
generator.add(BatchNormalization()) # 데이터의 정규화 작업 : 분산 0 , 평균 1
generator.add(Reshape((7,7,128))) # Conv 에 입력 가능한 자료로 변환
generator.add(UpSampling2D()) # 이미지 확대 : 14 * 14 # 이 레이어를 지날떄 14 *14 가 됨
generator.add(Conv2D(64,kernel_size = 5,padding = 'same')) # 14 * 14 size 변동 없음
generator.add(BatchNormalization()) # 데이터 정규화
generator.add(Activation(LeakyReLU(0.2))) # 활성화 함수 는 LeakyReLU 사용
generator.add(UpSampling2D()) # 이미지 확대 28 * 28 # 이 레이어를 지날떄 28*28 이 됨
generator.add(Conv2D(1,kernel_size = 5,padding='same',activation = 'tanh'))
generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              633472    
                                                                 
 batch_normalization_3 (Batc  (None, 6272)             25088     
 hNormalization)                                                 
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d_2 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        204864    
                                                                 
 batch_normalization_4 (Batc  (None, 14, 14, 64)      

In [10]:
# 판별자 모델 생성
discriminator = Sequential()
discriminator.add(Conv2D(64,kernel_size = 5 , strides = 2, input_shape = (28,28,1),padding = 'same')) # 28 * 28 * 64
discriminator.add(Activation(LeakyReLU(0.2))) # 활성화 함수 지정
discriminator.add(Dropout(0.3)) # 30% node 꺼줌
discriminator.add(Conv2D(128,kernel_size = 5 , strides = 2 , padding = 'same')) # 28 * 28 * 128
discriminator.add(Activation(LeakyReLU(0.2))) # 활성화 함수 지정
discriminator.add(Dropout(0.3)) # 30 % node  꺼줌
discriminator.add(Flatten()) # 1차원으로 평탄화
discriminator.add(Dense(1,activation = 'sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer ='adam')
discriminator.traunable = False
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 14, 14, 64)        1664      
                                                                 
 activation_4 (Activation)   (None, 14, 14, 64)        0         
                                                                 
 dropout_2 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 activation_5 (Activation)   (None, 7, 7, 128)         0         
                                                                 
 dropout_3 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)             

In [14]:
from tensorflow.keras.datasets import mnist

(X_train,y_train),(_,_) = mnist.load_data()

X_train.shape


(60000, 28, 28)

In [18]:
# 생성자와 판별자 모델을 연결시키는 모델 gan 생성
ginput = Input(shape = (100,))
dis_output = discriminator(generator(ginput))
gan = Model(ginput,dis_output)
gan.compile(loss = 'binary_crossentropy',optimizer = 'adam')
gan.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_4 (Sequential)   (None, 1)                 212865    
                                                                 
Total params: 1,078,146
Trainable params: 1,065,474
Non-trainable params: 12,672
_________________________________________________________________


In [ ]:
# 학습하는 함수 생성
def gan_train(epoch,batch_size,saving_interval):
    # MNIST_data 불러오기
    (X_train,_),(_,_) = mnist.load_data()